# Аналитический кейс: анализ клиентской базы фитнес-сервиса

Цель проекта — проанализировать поведение клиентов фитнес-сервиса, выявить ключевые сегменты пользователей,
оценить их вклад в активность и выручку, а также изучить динамику клиентской базы во времени.

Анализ выполнен с использованием Python и библиотек pandas, numpy и matplotlib.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Данные и подготовка 

В анализе используются обезличенные данные о клиентах фитнес-сервиса, их тренировках и посещениях.
На этом этапе выполняется загрузка данных и первичная подготовка для анализа.

In [ ]:
# загружаем очищенный датасет
df = pd.read_csv('merged_data.csv')
df.head()

## Клиенты с преобладанием индивидуальных тренировок

В этом разделе анализируются клиенты, у которых более 70% тренировок проходят в индивидуальном формате.
Пытаемся понять, отличается ли их поведение и ценность от остальных пользователей.

In [ ]:
df['share_indiv'] = df['cnt_indiv'] / df['cnt_total']
df.head(10)

In [ ]:
share_70_indiv = df.loc[df['share_indiv'] > 0.7].copy()
share_70_indiv.head(10)

In [ ]:
avg_check_all = df['sum_total'].sum() / df['cnt_total'].sum()
avg_check = share_70_indiv['sum_total'].sum() / share_70_indiv['cnt_total'].sum()
print(avg_check_all)
print(avg_check)

In [ ]:
users_stats = share_70_indiv.groupby('id_user').agg(total_train = ('cnt_total', 'sum'), nmonths = ('mon', 'count')).reset_index()
users_stats.head(10)

In [ ]:
users_stats['avg_train'] = users_stats['total_train'] / users_stats['nmonths']
users_stats.head(10)

In [ ]:
avg_total_train = users_stats['avg_train'].mean()
print(avg_total_train)

In [ ]:
users_stats = share_70_indiv.groupby(['city', 'gender']).agg(cnt_users = ('id_user', 'nunique')).reset_index()
users_stats

В Москве и СПб женщины чаще выбирают индивидуальные тренировки. Особенно выражена разница  в Москве (в 2 раза). 
Средний чек по таким клиентам сильно выше, чем средний чек по всем клиентам (1751р против 1058р). 

## Самые активные клиенты

Выделена группа наиболее активных клиентов по количеству посещений.
Проанализировано их распределение по полу и городам, а также вклад в общую активность сервиса.

In [ ]:
df = pd.read_csv('merged_data.csv')

In [ ]:
top_10_train = df.groupby(['id_user']).agg({'cnt_total': 'sum'}).reset_index().sort_values('cnt_total', ascending = False).head(10)
top_10_train

In [ ]:
user_lst = top_10_train['id_user'].tolist()
user_lst

In [ ]:
df_top_10 = df[df['id_user'].isin(user_lst)]
df_top_10.head()

In [ ]:
df_top_10['id_user'].nunique()

In [ ]:
city_gender_stats = df_top_10.groupby(['city', 'gender']).agg(nusers = ('id_user', 'nunique')).reset_index()
city_gender_stats

Самые активные клиенты из Москвы, поровну мужчин и женщин.

## Динамика клиентской базы

Проведен анализ изменения клиентской базы по месяцам.
Рассматриваются приток новых клиентов, отток и общее число активных пользователей во времени.

In [ ]:
df = pd.read_csv('merged_data.csv')
df.head()

In [ ]:
new_df = df[df['mon'] == df['min_mon']].groupby('mon').agg(new_clients = ('id_user', 'nunique')).reset_index()
new_df

In [ ]:
gone_df = df[df['mon'] == df['max_mon']].groupby('mon').agg(gone_clients = ('id_user', 'nunique')).reset_index()
gone_df

In [ ]:
active_df = df.groupby('mon').agg(active_clients = ('id_user', 'nunique')).reset_index()
active_df

In [ ]:
monthly_stats = active_df.merge(new_df, on = 'mon', how = 'left').merge(gone_df, on = 'mon', how = 'left') 
monthly_stats

In [ ]:
plt.figure(figsize = (10,5))

plt.plot(monthly_stats['mon'], monthly_stats['active_clients'], label = 'active_clients')
plt.plot(monthly_stats['mon'], monthly_stats['new_clients'], label = 'new_clients')
plt.plot(monthly_stats['mon'], monthly_stats['gone_clients'], label = 'gone_clients')

plt.xticks(rotation = 45)
plt.grid(True)
plt.legend()
plt.xlabel('Месяцы')
plt.ylabel('Количество клиентов')
plt.title('Клиентская активность по месяцам')

plt.show 

## Выводы

- Клиенты с высокой долей индивидуальных тренировок демонстрируют более высокий уровень вовлеченности и среднего чека.
- Самые активные пользователи вносят существенный вклад в общую активность сервиса.
- Основная концентрация активных клиентов наблюдается в крупных городах.
- Анализ динамики по месяцам позволяет выявлять периоды роста и снижения пользовательской активности.

Кейс демонстрирует применение Python для прикладного анализа клиентского поведения и сегментации пользователей.